In [1]:
import warnings
warnings.filterwarnings('ignore')
from fastai.imports import *
from fastai.structured import *
from pandas_summary import DataFrameSummary
%matplotlib inline 
from sklearn.linear_model import LogisticRegression
from fancyimpute import  KNN, NuclearNormMinimization, SoftImpute
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from catboost import *
import fancyimpute  
from sklearn.preprocessing import Imputer
import lightgbm as lgb

Using TensorFlow backend.


In [19]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
del train['train_id']
id = test['test_id']
del test['test_id']
train.shape, test.shape

((18255, 1234), (27285, 1233))

In [20]:
col = train.isnull().sum().sort_values(ascending = False)<0.50*train.shape[0]
train = train.loc[:,col]
train.head(3)

,AA3,AA4,AA6,AA7,AA14,AA15,DG1,is_female,DG3,DG3A,...,LN2_2,LN2_3,LN2_4,LN2_RIndLngBEOth,LN2_WIndLngBEOth,GN1,GN2,GN3,GN4,GN5
0,3,32,NaN,323011,3854,481,1975,1,3,4,...,1,1,1,NaN,NaN,99.0,99,99,99,99
1,2,26,8.0,268131,2441,344,1981,1,8,4,...,1,3,4,Bengali,Bengali,NaN,1,2,2,2
2,1,16,7.0,167581,754,143,1995,1,3,2,...,1,2,2,Hindi,Hindi,1.0,2,2,2,2


In [21]:
index = col[(col == True)].index
test = test.loc[:,index]
del test['is_female']
test.shape, train.shape

((27285, 360), (18255, 361))

In [24]:
train.dropna(inplace=True), train.shape

(None, (42, 361))

In [23]:
col2 = list(train.select_dtypes(include=['object']).columns)
col2

['LN2_RIndLngBEOth', 'LN2_WIndLngBEOth']

In [ ]:
y = train['is_female']
X = train.drop(['is_female'], axis = 1).fillna(0)
model = xgb.XGBClassifier(max_depth=20, min_child_weight=12,  n_estimators=100,\
                          n_jobs=-1 , verbose=1,learning_rate=0.07)
# model.fit(X,y)
cate_features_index = [
 'LN2_RIndLngBEOth',
 'LN2_WIndLngBEOth']
clf = CatBoostClassifier(eval_metric="AUC",one_hot_max_size=10, \
                        depth=10, iterations= 300, l2_leaf_reg= 5, learning_rate= 0.08)
# clf.fit(X,y, cat_features=cate_features_index)
params = {}
params['learning_rate'] = 0.045
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'auc'
params['sub_feature'] = 0.3
params['num_leaves'] = 20
params['max_depth'] = 15
params["n_estimators"] = 300
d_train = lgb.Dataset(X, label=y)
lg = lgb.train( params,d_train, 300,categorical_feature=cate_features_index)

### Convert all float into int

In [39]:
col2 = list(train.select_dtypes(include=['object']).columns)
col2

['LN2_RIndLngBEOth', 'LN2_WIndLngBEOth']

In [41]:
for item in col2:
    train[item] = train[item].astype("category")

In [40]:
train.loc[:,col2] = train.loc[:,col2].fillna("missing")
test.loc[:,col2] = test.loc[:,col2].fillna("missing")

In [185]:
train["row_nulls"] = train.isnull().sum(axis=1)/float(train.shape[1])
test["row_nulls"] = test.isnull().sum(axis=1)/float(test.shape[1])
# train.fillna(9999,inplace=True)
# test.fillna(9999,inplace=True)

In [ ]:
# model = Imputer(strategy="most_frequent")
# train = model.fit_transform(train)
# test = model.transform(test)
train.fillna(train.mode().iloc[0], inplace=True)
test.fillna(test.mode().iloc[0], inplace=True)

In [187]:
float_col = list(train.select_dtypes(include=['float64']).columns)

In [188]:
train[float_col] = train[float_col].astype("int64")
# test["is_female"] = 0
test[float_col] = test[float_col].astype("int64")
# del test["is_female"]

In [189]:
train["row_nulls"] = train["row_nulls"].astype("float64")
test["row_nulls"] = test["row_nulls"].astype("float64")
train.head(3)

,AA3,AA4,AA5,AA6,AA7,AA14,AA15,DG1,is_female,DG3,...,LN2_3,LN2_4,LN2_RIndLngBEOth,LN2_WIndLngBEOth,GN1,GN2,GN3,GN4,GN5,row_nulls
0,3,32,3,6,323011,3854,481,1975,1,3,...,1,1,missing,missing,99,99,99,99,99,0.0
1,2,26,3,8,268131,2441,344,1981,1,8,...,3,4,Bengali,Bengali,1,1,2,2,2,0.0
2,1,16,3,7,167581,754,143,1995,1,3,...,2,2,Hindi,Hindi,1,2,2,2,2,0.0


In [190]:
col = list(train.select_dtypes(include=['object','int64']).columns)
v = (train[col].apply(lambda x: x.nunique(), axis=0)>3).reset_index()
col_filtered = list(v.loc[(v.iloc[:,1]==True),'index'])
u = (train[col_filtered].nunique()<100).reset_index()
col_filtered = list(u.loc[(u.iloc[:,1]==True),'index'])

### Target mean encoding on col_filtered 

In [191]:
from sklearn.model_selection import KFold
def reg_target_encoding(train, col , splits=5):
    """ Computes regularize mean encoding.
    Inputs:
       train: training dataframe
       
    """
    train2 = train.copy()
    # YOUR CODE HERE
    col_name = col + '_mean_enc'
    kf = KFold(n_splits = splits, shuffle = False, random_state = 2)
    s = kf.split(train2)
    for i in range(splits):
        train_index, test_index = next(s)
        mean_device_type = train2.loc[train_index].groupby(col).is_female.mean()
        train2.loc[test_index,col_name] = train2[col].map(mean_device_type)
    global_mean = train2.is_female.mean()
    train2[col_name].fillna(global_mean, inplace=True)
    return train2
    #raise NotImplementedError()

In [2]:
def mean_encoding(df, test, col):
    for item in col:
        mean_device_type = df.groupby(item).is_female.mean()
        test[item+"_mean_enc"] = test[item].map(mean_device_type)
        df[item+"_mean_enc"] = df[item].map(mean_device_type)
    return df,test

In [198]:
def process_data(train,test):
    combine = pd.concat([train,test])
    object_cols = list(test.select_dtypes(include=['object','category']).columns)
    for c in object_cols:
        combine[c] = combine[c].astype('category').cat.codes+1  
    train = combine.iloc[:train.shape[0],]
    test = combine.iloc[train.shape[0]:,]
    return train,test

In [199]:
train,test = mean_encoding(train, test, col_filtered)
train,test = process_data(train,test)

In [200]:
global_mean = train.is_female.mean()
test.fillna(global_mean, inplace=True)

In [201]:
del test['is_female']
test.head(3), train.shape

(   AA14  AA15  AA3  AA3_mean_enc  AA4  AA4_mean_enc  AA5  AA5_mean_enc  AA6  \
 0  4479   535    4      0.541462   41      0.533381    3      0.538318    7   
 1  3803   476    3      0.476144   32      0.446961    2      0.524664    6   
 2  5610   585    3      0.476144   36      0.442308    5      0.528686    6   
 
       AA7    ...      MT6C  MT6C_mean_enc  MT6_mean_enc  MT7  MT7A  \
 0  417211    ...         0       0.576092      0.499081    2     1   
 1  322011    ...         0       0.576092      0.499081    2     1   
 2  365011    ...         0       0.576092      0.499081    2     1   
 
    MT7A_mean_enc  MT8  MT9  MT9_mean_enc  row_nulls  
 0       0.526598    2   12      0.696429        0.0  
 1       0.526598    2   15      0.486773        0.0  
 2       0.526598    2   13      0.741351        0.0  
 
 [3 rows x 903 columns], (18255, 904))

In [77]:
# mn.bar(test)

In [78]:
# train_filled_knn = KNN(k=3).complete(train)
# train_filled_knn = pd.DataFrame(train_filled_knn, columns=train.columns, index = train.index)
# train_filled_knn.shape

In [79]:
# test_filled_knn = KNN(k=3).complete(test)
# test_filled_knn = pd.DataFrame(test_filled_knn, columns=test.columns, index = test.index)
# test_filled_knn.shape

In [202]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [203]:
y = train['is_female']
X = train.drop(['is_female'], axis = 1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=10)

In [204]:
def auc(m, train, val): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
                            metrics.roc_auc_score(y_val,m.predict_proba(val)[:,1]))

### Parameter Tuning Random Search XGBoost

In [ ]:
model = xgb.XGBClassifier()
param_dist = {"max_depth": [20,25,30],
              "min_child_weight" : [8,10,12],
              "learning_rate": [0.07, 0.1,0.2],
               }

# run randomized search
random_search = GridSearchCV(model, param_grid=param_dist, cv = 4, 
                                   verbose=5, n_jobs=-1)
random_search.fit(X, y)

In [178]:
random_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.07, max_delta_step=0,
       max_depth=25, min_child_weight=12, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

### Fitting Final Model XGBoost

In [83]:
model = xgb.XGBClassifier(max_depth=20, min_child_weight=12,  n_estimators=100,\
                          n_jobs=-1 , verbose=1,learning_rate=0.07)
model.fit(X_train,y_train)

auc(model, X_train, X_val)

(0.9954960762467275, 0.9711507973037938)

In [84]:
model.fit(X,y)
pred = model.predict_proba(test)

In [85]:
submit = pd.DataFrame()
submit['test_id'] = id
submit['is_female'] = pred[:,1]
submit.to_csv("submissiongb_971.csv", index=False)

In [114]:
# test.loc[:,'is_female'] = pred[:,1]
# test.loc[(test['DL2_96_OTHERS']=="House wife"),'is_female']
# test['DL2_96_OTHERS']=="House wife"
# (train[col_filtered].nunique()>10).reset_index()

#### Parameters for xgb - 
params = {}

params['learning_rate'] = 0.07

params['min_child_weight'] = 12

params['max_depth'] = 20

params["n_estimators"] = 100

col = train.isnull().sum().sort_values(ascending = False)<0.99*train.shape[0]

v = (train[col].apply(lambda x: x.nunique(), axis=0)>3).reset_index()

u = (train[col_filtered].nunique()<100).reset_index()

train.fillna(9999,inplace=True)

In [28]:
xgb_pars = {"min_child_weight": 12, "eta": 0.07, "max_depth": 20,
           "eval_metric": "logloss"}

d_train = xgb.DMatrix(X_train, label=y_train)
d_val = xgb.DMatrix(X_val, label=y_val)

watchlist = [(d_train, 'train'), (d_val, 'valid')]

bst = xgb.train(xgb_pars, d_train, 600, watchlist, early_stopping_rounds=100, verbose_eval=50)

[0]	train-logloss:0.639262	valid-logloss:0.644994
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 100 rounds.
[50]	train-logloss:0.066105	valid-logloss:0.307764
[100]	train-logloss:0.032794	valid-logloss:0.388873
Stopping. Best iteration:
[38]	train-logloss:0.095801	valid-logloss:0.243603



### Catboost Classifier

In [74]:
cate_features_index = np.where(X.dtypes != "float")[0]

In [75]:
# train default catBoost classifier. Default loss metric is LogLoss (lower is better)
clf = CatBoostClassifier(eval_metric="AUC",one_hot_max_size=10, \
                            depth=10, iterations= 300, l2_leaf_reg= 5, learning_rate= 0.08)

In [ ]:
# clf.fit(X,Y , cat_features=cat_dims)
clf.fit(X_train,y_train, cat_features=cate_features_index)

In [ ]:
auc(clf, X_train, X_val)

In [ ]:
clf.fit(X,y, cat_features=cate_features_index)

In [80]:
pred = clf.predict_proba(test)

In [82]:
submit = pd.DataFrame()
submit['test_id'] = id
submit['is_female'] = pred[:,1]
submit.to_csv("submissionclf_0.97.csv", index=False)

### Parameter Tuning

In [55]:
from sortedcontainers import SortedList
import copy
import collections
import numpy as np
from itertools import product,chain
import pandas
from sklearn.model_selection import KFold
import catboost as cb

''' a class for doing grid search on a set of parameters provided in a dict. 'pdict' should be a dictionary like the following:
pdict = {'depth':[1,2], 'iterations':[250,100,500], 'thread_count':4}

when grid_search is called it will return an iterator that provides samples from the dictionary e.g.
{'depth':1, 'iterations':250, 'thread_count':4}
{'depth':2, 'iterations':250, 'thread_count':4}
{'depth':1, 'iterations':100, 'thread_count':4}
etc.
after calling an iteration of grid_search, you need to test the classifier and run 'register_result'
This will update the internal list of results, so that the next call to grid_search will use the best
parameters for all the parameters not currently being updated.

grid_search can be provided a list e.g. grid_search(['depth']) this will use the current best parameters for all
the other arguments and only search over 'depth'. You can then call e.g. grid_search(['iterations']) and it will use
the best depth found previously and cycle through all the 'iterations'. Searching incrementally can be much faster
than doing a full grid search, but may miss the global optimum. '''
class paramsearch:
    def __init__(self,pdict):    
        self.pdict = {}
        # if something is not passed in as a sequence, make it a sequence with 1 element
        #   don't treat strings as sequences
        for a,b in pdict.items():
            if isinstance(b, collections.Sequence) and not isinstance(b, str): self.pdict[a] = b
            else: self.pdict[a] = [b]
        # our results are a sorted list, so the best score is always the final element
        self.results = SortedList()       
                    
    def grid_search(self,keys=None):
        # do grid search on only the keys listed. If none provided, do all
        if keys==None: keylist = self.pdict.keys()
        else: keylist = keys
 
        listoflists = [] # this will be list of lists of key,value pairs
        for key in keylist: listoflists.append([(key,i) for i in self.pdict[key]])
        for p in product(*listoflists):
            # do any changes to the current best parameter set
            if len(self.results)>0: template = self.results[-1][1]
            else: template = {a:b[0] for a,b in self.pdict.items()}
            # if our updates are the same as current best, don't bother
            if self.equaldict(dict(p),template): continue
            # take the current best and update just the ones to change
            yield self.overwritedict(dict(p),template)
                              
    def equaldict(self,a,b):
        for key in a.keys(): 
            if a[key] != b[key]: return False
        return True            
                              
    def overwritedict(self,new,old):
        old = copy.deepcopy(old)
        for key in new.keys(): old[key] = new[key]
        return old            
    
    # save a (score,params) pair to results. Since 'results' is a sorted list,
    #   the best score is always the final element. A small amount of noise is added
    #   because sorted lists don't like it when two scores are exactly the same    
    def register_result(self,result,params):
        self.results.add((result+np.random.randn()*1e-10,params))    
        
    def bestscore(self):
        return self.results[-1][0]
        
    def bestparam(self):
        return self.results[-1][1]

In [56]:
params = { 'iterations':[300],
          'learning_rate':[0.05,0.08,0.1],
         "one_hot_max_size":[5,10,15,20]}

In [57]:
# this function does 3-fold crossvalidation with catboostclassifier          
def crossvaltest(params,train_set,train_label,cat_dims,n_splits=3):
    kf = KFold(n_splits=n_splits,shuffle=True) 
    res = []
    for train_index, test_index in kf.split(train_set):
        train = train_set.iloc[train_index,:]
        test = train_set.iloc[test_index,:]

        labels = train_label.ix[train_index]
        test_labels = train_label.ix[test_index]

        clf = cb.CatBoostClassifier(**params)
        clf.fit(train, np.ravel(labels), cat_features=cat_dims)

        res.append(np.mean(clf.predict(test)==np.ravel(test_labels)))
    return np.mean(res)

In [58]:
# this function runs grid search on several parameters
def catboost_param_tune(params,train_set,train_label,cat_dims=None,n_splits=3):
    ps = paramsearch(params)
    # search 'border_count', 'l2_leaf_reg' etc. individually 
    #   but 'iterations','learning_rate' together
    for prms in chain(
                      ps.grid_search(['iterations','learning_rate']),
                      ps.grid_search(['one_hot_max_size'])):
        res = crossvaltest(prms,train_set,train_label,cat_dims,n_splits)
        # save the crossvalidation result so that future iterations can reuse the best parameters
        ps.register_result(res,prms)
        print(res,prms,'best:',ps.bestscore(),ps.bestparam())
    return ps.bestparam()

bestparams = catboost_param_tune(params,X_train,y_train,cat_dims =cate_features_index)

0:	learn: 0.6918608	total: 295ms	remaining: 1m 28s
1:	learn: 0.6905714	total: 1.11s	remaining: 2m 46s
2:	learn: 0.6894217	total: 2.19s	remaining: 3m 36s
3:	learn: 0.6884782	total: 3.31s	remaining: 4m 4s
4:	learn: 0.6876009	total: 4.18s	remaining: 4m 6s
5:	learn: 0.6870186	total: 4.42s	remaining: 3m 36s
6:	learn: 0.6863471	total: 5.29s	remaining: 3m 41s
7:	learn: 0.6859428	total: 5.54s	remaining: 3m 22s
8:	learn: 0.6856006	total: 5.92s	remaining: 3m 11s
9:	learn: 0.6852105	total: 6.51s	remaining: 3m 8s
10:	learn: 0.6845186	total: 7.71s	remaining: 3m 22s
11:	learn: 0.6841421	total: 8.51s	remaining: 3m 24s
12:	learn: 0.6837613	total: 9.11s	remaining: 3m 21s
13:	learn: 0.6831828	total: 9.58s	remaining: 3m 15s
14:	learn: 0.6828478	total: 10.3s	remaining: 3m 16s
15:	learn: 0.6827438	total: 10.4s	remaining: 3m 4s
16:	learn: 0.6824345	total: 11s	remaining: 3m 3s
17:	learn: 0.6821722	total: 11.3s	remaining: 2m 57s
18:	learn: 0.6818935	total: 12.2s	remaining: 3m
19:	learn: 0.6818347	total: 12.5s

158:	learn: 0.6482111	total: 2m 4s	remaining: 1m 50s
159:	learn: 0.6480750	total: 2m 5s	remaining: 1m 49s
160:	learn: 0.6478555	total: 2m 6s	remaining: 1m 49s
161:	learn: 0.6475930	total: 2m 6s	remaining: 1m 48s
162:	learn: 0.6472836	total: 2m 7s	remaining: 1m 47s
163:	learn: 0.6468448	total: 2m 9s	remaining: 1m 47s
164:	learn: 0.6466415	total: 2m 9s	remaining: 1m 46s
165:	learn: 0.6460653	total: 2m 10s	remaining: 1m 45s
166:	learn: 0.6457859	total: 2m 11s	remaining: 1m 44s
167:	learn: 0.6456146	total: 2m 11s	remaining: 1m 43s
168:	learn: 0.6451522	total: 2m 12s	remaining: 1m 42s
169:	learn: 0.6446536	total: 2m 13s	remaining: 1m 42s
170:	learn: 0.6443830	total: 2m 14s	remaining: 1m 41s
171:	learn: 0.6442335	total: 2m 15s	remaining: 1m 40s
172:	learn: 0.6440423	total: 2m 16s	remaining: 1m 39s
173:	learn: 0.6436862	total: 2m 16s	remaining: 1m 39s
174:	learn: 0.6434599	total: 2m 18s	remaining: 1m 38s
175:	learn: 0.6431738	total: 2m 19s	remaining: 1m 38s
176:	learn: 0.6426952	total: 2m 20s

13:	learn: 0.6824460	total: 6.72s	remaining: 2m 17s
14:	learn: 0.6822147	total: 6.99s	remaining: 2m 12s
15:	learn: 0.6813685	total: 7.62s	remaining: 2m 15s
16:	learn: 0.6812599	total: 7.69s	remaining: 2m 7s
17:	learn: 0.6809688	total: 8.33s	remaining: 2m 10s
18:	learn: 0.6807516	total: 8.99s	remaining: 2m 12s
19:	learn: 0.6806860	total: 9.06s	remaining: 2m 6s
20:	learn: 0.6806305	total: 9.13s	remaining: 2m 1s
21:	learn: 0.6802714	total: 9.79s	remaining: 2m 3s
22:	learn: 0.6797319	total: 10.5s	remaining: 2m 5s
23:	learn: 0.6791522	total: 11s	remaining: 2m 6s
24:	learn: 0.6788100	total: 11.7s	remaining: 2m 8s
25:	learn: 0.6785349	total: 12.4s	remaining: 2m 10s
26:	learn: 0.6782829	total: 13.1s	remaining: 2m 12s
27:	learn: 0.6781168	total: 13.8s	remaining: 2m 14s
28:	learn: 0.6776565	total: 14.6s	remaining: 2m 16s
29:	learn: 0.6774274	total: 15.3s	remaining: 2m 17s
30:	learn: 0.6769943	total: 16.1s	remaining: 2m 19s
31:	learn: 0.6764665	total: 16.9s	remaining: 2m 21s
32:	learn: 0.6764219	

170:	learn: 0.6467908	total: 1m 53s	remaining: 1m 25s
171:	learn: 0.6463901	total: 1m 53s	remaining: 1m 24s
172:	learn: 0.6461557	total: 1m 54s	remaining: 1m 24s
173:	learn: 0.6458207	total: 1m 55s	remaining: 1m 23s
174:	learn: 0.6457203	total: 1m 56s	remaining: 1m 22s
175:	learn: 0.6452465	total: 1m 56s	remaining: 1m 22s
176:	learn: 0.6452361	total: 1m 57s	remaining: 1m 21s
177:	learn: 0.6452289	total: 1m 58s	remaining: 1m 21s
178:	learn: 0.6450460	total: 2m	remaining: 1m 21s
179:	learn: 0.6447395	total: 2m 1s	remaining: 1m 20s
180:	learn: 0.6445645	total: 2m 2s	remaining: 1m 20s
181:	learn: 0.6444505	total: 2m 3s	remaining: 1m 20s
182:	learn: 0.6440218	total: 2m 4s	remaining: 1m 19s
183:	learn: 0.6438059	total: 2m 5s	remaining: 1m 19s
184:	learn: 0.6438004	total: 2m 7s	remaining: 1m 18s
185:	learn: 0.6434893	total: 2m 7s	remaining: 1m 18s
186:	learn: 0.6434749	total: 2m 9s	remaining: 1m 17s
187:	learn: 0.6433209	total: 2m 10s	remaining: 1m 17s
188:	learn: 0.6432369	total: 2m 11s	rema

26:	learn: 0.6777352	total: 17.9s	remaining: 3m
27:	learn: 0.6774306	total: 18.7s	remaining: 3m 1s
28:	learn: 0.6771879	total: 19.6s	remaining: 3m 3s
29:	learn: 0.6770515	total: 20.5s	remaining: 3m 4s
30:	learn: 0.6768324	total: 21.5s	remaining: 3m 6s
31:	learn: 0.6765733	total: 22.3s	remaining: 3m 6s
32:	learn: 0.6762414	total: 23s	remaining: 3m 5s
33:	learn: 0.6761339	total: 23.7s	remaining: 3m 5s
34:	learn: 0.6756788	total: 24.4s	remaining: 3m 4s
35:	learn: 0.6753828	total: 25.1s	remaining: 3m 4s
36:	learn: 0.6749117	total: 26s	remaining: 3m 4s
37:	learn: 0.6745153	total: 26.7s	remaining: 3m 4s
38:	learn: 0.6744496	total: 27.4s	remaining: 3m 3s
39:	learn: 0.6742795	total: 28.4s	remaining: 3m 4s
40:	learn: 0.6739388	total: 29.2s	remaining: 3m 4s
41:	learn: 0.6736649	total: 30.1s	remaining: 3m 5s
42:	learn: 0.6734617	total: 31.1s	remaining: 3m 6s
43:	learn: 0.6732414	total: 32.1s	remaining: 3m 6s
44:	learn: 0.6729364	total: 33s	remaining: 3m 6s
45:	learn: 0.6727048	total: 33.7s	remain

...

161:	learn: 0.6183957	total: 50s	remaining: 42.6s
162:	learn: 0.6178901	total: 50.5s	remaining: 42.4s
163:	learn: 0.6174018	total: 51s	remaining: 42.3s
164:	learn: 0.6169212	total: 51.4s	remaining: 42s
165:	learn: 0.6164899	total: 51.7s	remaining: 41.7s
166:	learn: 0.6160933	total: 52.1s	remaining: 41.5s
167:	learn: 0.6158228	total: 52.4s	remaining: 41.2s
168:	learn: 0.6154403	total: 53s	remaining: 41.1s
169:	learn: 0.6146738	total: 53.3s	remaining: 40.8s
170:	learn: 0.6145190	total: 53.6s	remaining: 40.4s
171:	learn: 0.6141556	total: 53.9s	remaining: 40.1s
172:	learn: 0.6137502	total: 54.1s	remaining: 39.7s
173:	learn: 0.6134067	total: 54.4s	remaining: 39.4s
174:	learn: 0.6130314	total: 54.6s	remaining: 39s
175:	learn: 0.6127464	total: 54.9s	remaining: 38.7s
176:	learn: 0.6124897	total: 55.3s	remaining: 38.4s
177:	learn: 0.6121861	total: 55.5s	remaining: 38.1s
178:	learn: 0.6119363	total: 55.8s	remaining: 37.7s
179:	learn: 0.6113687	total: 56.1s	remaining: 37.4s
180:	learn: 0.6112681	

19:	learn: 0.6774013	total: 5.05s	remaining: 1m 10s
20:	learn: 0.6766493	total: 5.29s	remaining: 1m 10s
21:	learn: 0.6764852	total: 5.59s	remaining: 1m 10s
22:	learn: 0.6760871	total: 5.88s	remaining: 1m 10s
23:	learn: 0.6756422	total: 6.22s	remaining: 1m 11s
24:	learn: 0.6749597	total: 6.58s	remaining: 1m 12s
25:	learn: 0.6745913	total: 6.83s	remaining: 1m 12s
26:	learn: 0.6744975	total: 6.95s	remaining: 1m 10s
27:	learn: 0.6739070	total: 7.24s	remaining: 1m 10s
28:	learn: 0.6736540	total: 7.59s	remaining: 1m 10s
29:	learn: 0.6734832	total: 7.98s	remaining: 1m 11s
30:	learn: 0.6730743	total: 8.28s	remaining: 1m 11s
31:	learn: 0.6723664	total: 8.57s	remaining: 1m 11s
32:	learn: 0.6720310	total: 8.89s	remaining: 1m 11s
33:	learn: 0.6713890	total: 9.16s	remaining: 1m 11s
34:	learn: 0.6709071	total: 9.42s	remaining: 1m 11s
35:	learn: 0.6701834	total: 9.69s	remaining: 1m 11s
36:	learn: 0.6697205	total: 9.95s	remaining: 1m 10s
37:	learn: 0.6691932	total: 10.2s	remaining: 1m 10s
38:	learn: 0

179:	learn: 0.6091531	total: 47s	remaining: 31.3s
180:	learn: 0.6089266	total: 47.3s	remaining: 31.1s
181:	learn: 0.6082944	total: 47.6s	remaining: 30.8s
182:	learn: 0.6078318	total: 47.9s	remaining: 30.6s
183:	learn: 0.6073957	total: 48.1s	remaining: 30.3s
184:	learn: 0.6070144	total: 48.4s	remaining: 30.1s
185:	learn: 0.6067922	total: 48.7s	remaining: 29.8s
186:	learn: 0.6062995	total: 49s	remaining: 29.6s
187:	learn: 0.6057412	total: 49.2s	remaining: 29.3s
188:	learn: 0.6049586	total: 49.5s	remaining: 29.1s
189:	learn: 0.6044579	total: 49.8s	remaining: 28.8s
190:	learn: 0.6040261	total: 50s	remaining: 28.6s
191:	learn: 0.6038482	total: 50.4s	remaining: 28.3s
192:	learn: 0.6035546	total: 50.6s	remaining: 28.1s
193:	learn: 0.6029872	total: 50.9s	remaining: 27.8s
194:	learn: 0.6022233	total: 51.1s	remaining: 27.5s
195:	learn: 0.6018911	total: 51.4s	remaining: 27.3s
196:	learn: 0.6017315	total: 51.7s	remaining: 27s
197:	learn: 0.6015006	total: 52s	remaining: 26.8s
198:	learn: 0.6012210	

38:	learn: 0.6678538	total: 9.67s	remaining: 1m 4s
39:	learn: 0.6672692	total: 9.95s	remaining: 1m 4s
40:	learn: 0.6666338	total: 10.2s	remaining: 1m 4s
41:	learn: 0.6663206	total: 10.5s	remaining: 1m 4s
42:	learn: 0.6657193	total: 10.8s	remaining: 1m 4s
43:	learn: 0.6653585	total: 11.1s	remaining: 1m 4s
44:	learn: 0.6649715	total: 11.3s	remaining: 1m 4s
45:	learn: 0.6647902	total: 11.6s	remaining: 1m 4s
46:	learn: 0.6641936	total: 11.8s	remaining: 1m 3s
47:	learn: 0.6635569	total: 12.1s	remaining: 1m 3s
48:	learn: 0.6632639	total: 12.4s	remaining: 1m 3s
49:	learn: 0.6629698	total: 12.6s	remaining: 1m 2s
50:	learn: 0.6624685	total: 12.8s	remaining: 1m 2s
51:	learn: 0.6620229	total: 13.1s	remaining: 1m 2s
52:	learn: 0.6618599	total: 13.4s	remaining: 1m 2s
53:	learn: 0.6618527	total: 13.5s	remaining: 1m 1s
54:	learn: 0.6613295	total: 13.7s	remaining: 1m
55:	learn: 0.6609061	total: 13.9s	remaining: 1m
56:	learn: 0.6607152	total: 14.2s	remaining: 1m
57:	learn: 0.6604273	total: 14.5s	remain

198:	learn: 0.6042423	total: 57.5s	remaining: 29.2s
199:	learn: 0.6038404	total: 57.8s	remaining: 28.9s
200:	learn: 0.6033214	total: 58.2s	remaining: 28.6s
201:	learn: 0.6030456	total: 58.6s	remaining: 28.4s
202:	learn: 0.6026365	total: 59s	remaining: 28.2s
203:	learn: 0.6023651	total: 59.5s	remaining: 28s
204:	learn: 0.6017945	total: 1m	remaining: 27.8s
205:	learn: 0.6012683	total: 1m	remaining: 27.6s
206:	learn: 0.6009732	total: 1m	remaining: 27.3s
207:	learn: 0.6006847	total: 1m 1s	remaining: 27.1s
208:	learn: 0.6003774	total: 1m 1s	remaining: 26.9s
209:	learn: 0.6000095	total: 1m 2s	remaining: 26.6s
210:	learn: 0.5994547	total: 1m 2s	remaining: 26.3s
211:	learn: 0.5987795	total: 1m 2s	remaining: 26.1s
212:	learn: 0.5984318	total: 1m 3s	remaining: 25.8s
213:	learn: 0.5976808	total: 1m 3s	remaining: 25.5s
214:	learn: 0.5975439	total: 1m 3s	remaining: 25.3s
215:	learn: 0.5973868	total: 1m 4s	remaining: 25s
216:	learn: 0.5971359	total: 1m 4s	remaining: 24.8s
217:	learn: 0.5968409	total

In [59]:
bestparams

{'iterations': 300, 'learning_rate': 0.1, 'one_hot_max_size': 20}

In [26]:
cate_features_index = np.where(X.dtypes != "float64")[0]

In [178]:
clf = CatBoostClassifier(eval_metric="AUC",one_hot_max_size=15, \
                        depth=10, iterations= 300, l2_leaf_reg= 5, learning_rate= 0.08)
clf.fit(X_train,y_train, cat_features=cate_features_index)
auc(clf, X_train, X_val)

0:	learn: 0.9408188	total: 616ms	remaining: 3m 4s
1:	learn: 0.9422923	total: 1.17s	remaining: 2m 53s
2:	learn: 0.9508683	total: 1.74s	remaining: 2m 51s
3:	learn: 0.9528166	total: 2.33s	remaining: 2m 52s
4:	learn: 0.9530156	total: 2.89s	remaining: 2m 50s
5:	learn: 0.9551370	total: 3.42s	remaining: 2m 47s
6:	learn: 0.9565702	total: 4s	remaining: 2m 47s
7:	learn: 0.9577884	total: 4.53s	remaining: 2m 45s
8:	learn: 0.9585896	total: 5.02s	remaining: 2m 42s
9:	learn: 0.9595881	total: 5.51s	remaining: 2m 39s
10:	learn: 0.9602762	total: 6.05s	remaining: 2m 38s
11:	learn: 0.9606178	total: 6.53s	remaining: 2m 36s
12:	learn: 0.9610906	total: 7.06s	remaining: 2m 35s
13:	learn: 0.9616433	total: 7.55s	remaining: 2m 34s
14:	learn: 0.9620046	total: 8.18s	remaining: 2m 35s
15:	learn: 0.9626651	total: 8.71s	remaining: 2m 34s
16:	learn: 0.9631600	total: 9.29s	remaining: 2m 34s
17:	learn: 0.9635917	total: 9.88s	remaining: 2m 34s
18:	learn: 0.9643029	total: 10.4s	remaining: 2m 34s
19:	learn: 0.9644338	total

157:	learn: 0.9851088	total: 1m 48s	remaining: 1m 37s
158:	learn: 0.9851898	total: 1m 49s	remaining: 1m 37s
159:	learn: 0.9852483	total: 1m 50s	remaining: 1m 36s
160:	learn: 0.9853989	total: 1m 50s	remaining: 1m 35s
161:	learn: 0.9854269	total: 1m 51s	remaining: 1m 35s
162:	learn: 0.9855852	total: 1m 52s	remaining: 1m 34s
163:	learn: 0.9857669	total: 1m 53s	remaining: 1m 33s
164:	learn: 0.9857929	total: 1m 53s	remaining: 1m 33s
165:	learn: 0.9858834	total: 1m 54s	remaining: 1m 32s
166:	learn: 0.9859493	total: 1m 54s	remaining: 1m 31s
167:	learn: 0.9859884	total: 1m 55s	remaining: 1m 30s
168:	learn: 0.9860764	total: 1m 56s	remaining: 1m 30s
169:	learn: 0.9860913	total: 1m 56s	remaining: 1m 29s
170:	learn: 0.9861802	total: 1m 57s	remaining: 1m 28s
171:	learn: 0.9862486	total: 1m 58s	remaining: 1m 27s
172:	learn: 0.9863217	total: 1m 58s	remaining: 1m 27s
173:	learn: 0.9864031	total: 1m 59s	remaining: 1m 26s
174:	learn: 0.9864259	total: 2m	remaining: 1m 25s
175:	learn: 0.9864424	total: 2m	

(0.9579858413863173, 0.9175019358751509)

In [28]:
pred_val_clf = clf.predict_proba(X_val)
X_val["predicted"] = pred_val_clf[:,1]
X_val.to_csv("pred_val_clf.csv", index=False)

In [89]:
clf.fit(X,y, cat_features=cate_features_index)
pred = clf.predict_proba(test)

0:	learn: 0.9492797	total: 758ms	remaining: 3m 46s
1:	learn: 0.9529351	total: 1.19s	remaining: 2m 57s
2:	learn: 0.9540952	total: 1.85s	remaining: 3m 3s
3:	learn: 0.9546328	total: 2.62s	remaining: 3m 13s
4:	learn: 0.9547116	total: 3.28s	remaining: 3m 13s
5:	learn: 0.9550021	total: 3.96s	remaining: 3m 14s
6:	learn: 0.9564878	total: 4.67s	remaining: 3m 15s
7:	learn: 0.9578466	total: 5.36s	remaining: 3m 15s
8:	learn: 0.9592753	total: 6.13s	remaining: 3m 18s
9:	learn: 0.9607268	total: 6.93s	remaining: 3m 20s
10:	learn: 0.9613514	total: 7.71s	remaining: 3m 22s
11:	learn: 0.9616263	total: 8.49s	remaining: 3m 23s
12:	learn: 0.9623532	total: 9.21s	remaining: 3m 23s
13:	learn: 0.9629434	total: 10s	remaining: 3m 24s
14:	learn: 0.9636179	total: 10.9s	remaining: 3m 26s
15:	learn: 0.9640335	total: 11.7s	remaining: 3m 28s
16:	learn: 0.9644730	total: 12.6s	remaining: 3m 29s
17:	learn: 0.9648977	total: 13.3s	remaining: 3m 28s
18:	learn: 0.9654495	total: 13.9s	remaining: 3m 25s
19:	learn: 0.9657788	tota

157:	learn: 0.9854313	total: 1m 54s	remaining: 1m 43s
158:	learn: 0.9854825	total: 1m 55s	remaining: 1m 42s
159:	learn: 0.9855049	total: 1m 56s	remaining: 1m 41s
160:	learn: 0.9855940	total: 1m 56s	remaining: 1m 40s
161:	learn: 0.9856239	total: 1m 57s	remaining: 1m 40s
162:	learn: 0.9857624	total: 1m 58s	remaining: 1m 39s
163:	learn: 0.9859243	total: 1m 59s	remaining: 1m 38s
164:	learn: 0.9860279	total: 2m	remaining: 1m 38s
165:	learn: 0.9860464	total: 2m	remaining: 1m 37s
166:	learn: 0.9862146	total: 2m 1s	remaining: 1m 36s
167:	learn: 0.9863560	total: 2m 2s	remaining: 1m 36s
168:	learn: 0.9864270	total: 2m 3s	remaining: 1m 35s
169:	learn: 0.9864549	total: 2m 3s	remaining: 1m 34s
170:	learn: 0.9864890	total: 2m 4s	remaining: 1m 34s
171:	learn: 0.9865899	total: 2m 5s	remaining: 1m 33s
172:	learn: 0.9866120	total: 2m 6s	remaining: 1m 32s
173:	learn: 0.9866519	total: 2m 6s	remaining: 1m 31s
174:	learn: 0.9867395	total: 2m 7s	remaining: 1m 31s
175:	learn: 0.9867548	total: 2m 8s	remaining:

In [91]:
submit = pd.DataFrame()
submit['test_id'] = id
submit['is_female'] = pred[:,1]
submit.to_csv("submissionclf_9715.csv", index=False)

#### Parameters for CLF - 
eval_metric="AUC",one_hot_max_size=15, \
                        depth=10, iterations= 300, l2_leaf_reg= 5, learning_rate= 0.08
    
    
col = train.isnull().sum().sort_values(ascending = False)<0.99*train.shape[0]

v = (train[col].apply(lambda x: x.nunique(), axis=0)>3).reset_index()

u = (train[col_filtered].nunique()<100).reset_index()

### LightGBM

In [205]:
import lightgbm as lgb

In [206]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# X_scaled = sc.fit_transform(X)
# X_scaled = pd.DataFrame(X_scaled, columns = X.columns)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=10)

In [207]:
# test_normalize = sc.transform(test)
# test_normalize = pd.DataFrame(test_normalize, columns = test.columns)

In [208]:
cate_features_index = np.where(X.dtypes != "float")[0]
len(list(cate_features_index))

699

In [165]:
lg = lgb.LGBMClassifier(silent=False)
param_dist = {"max_depth": [3, 5,15, 25],
              "learning_rate" : [0.07,0.1,0.15],
              'num_leaves': [10,20,50],
              "silent": [False]
             }
grid_search = GridSearchCV(lg, n_jobs=-1, param_grid=param_dist, cv = 5, scoring="roc_auc", verbose=5)
grid_search.fit(X_train,y_train)
grid_search.best_estimator_

In [209]:
d_train = lgb.Dataset(X, label=y)
params = {}
params['learning_rate'] = 0.045
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'auc'
params['sub_feature'] = 0.3
params['num_leaves'] = 20
params['max_depth'] = 15
params["n_estimators"] = 300
lg = lgb.train(params, d_train, 200)

In [210]:
def auc(m, train, val): 
    return (metrics.roc_auc_score(y_train,m.predict(train)),
                            metrics.roc_auc_score(y_val,m.predict(val)))

In [211]:
auc(lg, X_train, X_val)

(0.9902271995549364, 0.989813115095374)

In [174]:
d_train = lgb.Dataset(X, label=y)
lg = lgb.train(params, d_train, 300)
# pred_09716 = lg.predict(test)

In [177]:
submit = pd.DataFrame()
submit['test_id'] = id
pred = lg.predict(test)
submit['is_female'] = pred
submit.to_csv("submission_lgb_988.csv", index=False)

In [140]:
pred_val_lgb = lg.predict(X_val)
X_val["predicted"] = pred_val_lgb
X_val.to_csv("pred_val_lgb.csv", index=False)

#### Parameters for LGB - 
params = {}

params['learning_rate'] = 0.05

params['sub_feature'] = 0.3

params['num_leaves'] = 20

params['max_depth'] = 5

params["n_estimators"] = 400

col = train.isnull().sum().sort_values(ascending = False)<0.99*train.shape[0]

v = (train[col].apply(lambda x: x.nunique(), axis=0)>3).reset_index()

u = (train[col_filtered].nunique()<100).reset_index()

In [328]:
X_train.columns[cate_features_index]

Index(['AA14', 'AA15', 'AA3', 'AA4', 'AA5', 'AA6', 'AA7', 'DG1', 'DG10b',
       'DG10c',
       ...
       'MT5', 'MT6', 'MT6A', 'MT6B', 'MT6C', 'MT7', 'MT7A', 'MT8', 'MT9',
       'row_nulls'],
      dtype='object', length=700)

In [447]:
lgb.train??

### Ensembling

In [33]:
pred_mean = (pd.read_csv("submissions/stack3_coef_exchanged.csv")['is_female'] +\
             pd.read_csv("submissions/sub_xgb_int_21_ensemble_with_best.csv")['is_female'] +
             pd.read_csv("submissions/submission_ensemble.csv")['is_female'] + 
            pd.read_csv("submissions/submission_ensemble2.csv")['is_female'])/4

In [34]:
pred_mean = (pd.read_csv("submissions/sub_ensemble_with4_files.csv")['is_female'] +\
             pd.read_csv("submissions/submission_ensemble_9705.csv")['is_female'])/2

In [214]:
pred_mean = (pd.read_csv("submissions/submission2/sub_xgb_int_enc.csv")['is_female'] +\
             pd.read_csv("submissions/submission2/test_xgb_11_full_mode_imp.csv")['is_female'] )/2

In [92]:
pred_mean = (pd.read_csv("submissions/submission2/submission_ensemble3.csv")['is_female'] +\
             pd.read_csv("submissions/submission2/submission_ensemble2.csv")['is_female'] +\
             pd.read_csv("submissions/submission2/sub_xgb_int_21_ensemble_with_best.csv")['is_female']
            )/3

In [11]:
train.loc[train["DL1"] == 7, "is_female"].sum(), train.loc[train["DL1"] == 7, "is_female"].shape

(5531, (5664,))

In [34]:
pred_mean[list(test[test["DL1"] == 7].index)].sum(), pred_mean[list(test[test["DL1"] == 7].index)].shape

(8261.714046797188, (8438,))

In [77]:
((pred_mean[list(test[test["DL1"] == 7].index)]>0.7) & (pred_mean[list(test[test["DL1"] == 7].index)]<0.97)).sum()

360

In [108]:
pred_mean = pd.read_csv("submissions/submission2/sub_ensemble_with4_files.csv")['is_female']

In [109]:
ct =0
for i in range(test.shape[0]):
    if test.loc[i,"DL1"] == 7:
        if pred_mean[i]>0.75 and pred_mean[i]<0.95:
            ct = ct+1
            pred_mean[i] = round(np.random.uniform(0.998,0.999),16)

In [110]:
round(np.random.uniform(0.998,0.999),6)

0.998241

In [111]:
ct =0
for i in range(test.shape[0]):
    if test.loc[i,"DL2_96_OTHERS"] == "House wife":
        pred_mean[i] = round(np.random.uniform(0.999,0.9995),16)
        ct +=1

In [112]:
submit = pd.DataFrame()
submit['test_id'] = id
submit['is_female'] = pred_mean
submit.to_csv("submissions/submission2/submission_fiinal.csv", index=False)

In [107]:
ct

492

In [184]:
pred_mean3 = (pred[:,1] + pd.read_csv("sub_xgb_int_enc.csv")['is_female'] +\
             pd.read_csv("submissiongb_933.csv")['is_female'])/3
submit['is_female'] = pred_mean3
submit.to_csv("submission_ensemble2.csv", index=False)

### Random Forest

In [29]:
model = RandomForestClassifier()
param_dist = {"max_depth": [50,80,120],
              "max_features" : ["auto","sqrt","log2"],
              "min_samples_leaf": [5, 10,20],
              "n_estimators": [100]
               }

# run randomized search
random_search = GridSearchCV(model, param_grid=param_dist, cv = 4, 
                                   verbose=5, n_jobs=-1)
random_search.fit(X, y)

Fitting 4 folds for each of 27 candidates, totalling 108 fits
[CV] max_depth=50, max_features=auto, min_samples_leaf=5, n_estimators=100 
[CV] max_depth=50, max_features=auto, min_samples_leaf=5, n_estimators=100 
[CV] max_depth=50, max_features=auto, min_samples_leaf=5, n_estimators=100 
[CV] max_depth=50, max_features=auto, min_samples_leaf=5, n_estimators=100 
[CV]  max_depth=50, max_features=auto, min_samples_leaf=5, n_estimators=100, score=0.8902519167579409, total=   6.9s
[CV] max_depth=50, max_features=auto, min_samples_leaf=10, n_estimators=100 
[CV]  max_depth=50, max_features=auto, min_samples_leaf=5, n_estimators=100, score=0.8930762489044698, total=   7.0s
[CV] max_depth=50, max_features=auto, min_samples_leaf=10, n_estimators=100 
[CV]  max_depth=50, max_features=auto, min_samples_leaf=5, n_estimators=100, score=0.8996274380889766, total=   6.9s
[CV] max_depth=50, max_features=auto, min_samples_leaf=10, n_estimators=100 
[CV]  max_depth=50, max_features=auto, min_samples_l

[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   21.2s


[CV] max_depth=50, max_features=sqrt, min_samples_leaf=5, n_estimators=100 
[CV]  max_depth=50, max_features=auto, min_samples_leaf=20, n_estimators=100, score=0.883410037256191, total=   5.5s
[CV] max_depth=50, max_features=sqrt, min_samples_leaf=5, n_estimators=100 
[CV]  max_depth=50, max_features=sqrt, min_samples_leaf=5, n_estimators=100, score=0.8856516976998905, total=   8.9s
[CV] max_depth=50, max_features=sqrt, min_samples_leaf=10, n_estimators=100 
[CV]  max_depth=50, max_features=sqrt, min_samples_leaf=5, n_estimators=100, score=0.8919807186678352, total=   9.1s
[CV] max_depth=50, max_features=sqrt, min_samples_leaf=10, n_estimators=100 
[CV]  max_depth=50, max_features=sqrt, min_samples_leaf=5, n_estimators=100, score=0.8983125136971291, total=   9.1s
[CV] max_depth=50, max_features=sqrt, min_samples_leaf=10, n_estimators=100 
[CV]  max_depth=50, max_features=sqrt, min_samples_leaf=5, n_estimators=100, score=0.8926145079991233, total=   9.1s
[CV] max_depth=50, max_features=

[CV]  max_depth=80, max_features=sqrt, min_samples_leaf=10, n_estimators=100, score=0.8869412795793163, total=   8.9s
[CV] max_depth=80, max_features=sqrt, min_samples_leaf=20, n_estimators=100 
[CV]  max_depth=80, max_features=sqrt, min_samples_leaf=10, n_estimators=100, score=0.8948060486522025, total=   8.8s
[CV] max_depth=80, max_features=sqrt, min_samples_leaf=20, n_estimators=100 
[CV]  max_depth=80, max_features=sqrt, min_samples_leaf=10, n_estimators=100, score=0.8912995836072759, total=   8.7s
[CV] max_depth=80, max_features=sqrt, min_samples_leaf=20, n_estimators=100 
[CV]  max_depth=80, max_features=sqrt, min_samples_leaf=20, n_estimators=100, score=0.8751369112814896, total=   6.4s
[CV] max_depth=80, max_features=log2, min_samples_leaf=5, n_estimators=100 
[CV]  max_depth=80, max_features=sqrt, min_samples_leaf=20, n_estimators=100, score=0.8799298860648553, total=   6.0s
[CV] max_depth=80, max_features=log2, min_samples_leaf=5, n_estimators=100 
[CV]  max_depth=80, max_fea

[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  2.0min


[CV]  max_depth=80, max_features=log2, min_samples_leaf=10, n_estimators=100, score=0.8503833515881709, total=   4.7s
[CV] max_depth=80, max_features=log2, min_samples_leaf=20, n_estimators=100 
[CV]  max_depth=80, max_features=log2, min_samples_leaf=10, n_estimators=100, score=0.8549517966695881, total=   4.3s
[CV] max_depth=80, max_features=log2, min_samples_leaf=20, n_estimators=100 
[CV]  max_depth=80, max_features=log2, min_samples_leaf=10, n_estimators=100, score=0.8660968660968661, total=   4.1s
[CV] max_depth=80, max_features=log2, min_samples_leaf=20, n_estimators=100 
[CV]  max_depth=80, max_features=log2, min_samples_leaf=10, n_estimators=100, score=0.854043392504931, total=   3.9s
[CV] max_depth=80, max_features=log2, min_samples_leaf=20, n_estimators=100 
[CV]  max_depth=80, max_features=log2, min_samples_leaf=20, n_estimators=100, score=0.8328587075575027, total=   2.9s
[CV] max_depth=120, max_features=auto, min_samples_leaf=5, n_estimators=100 
[CV]  max_depth=80, max_fe

[CV]  max_depth=120, max_features=log2, min_samples_leaf=20, n_estimators=100, score=0.8533859303090072, total=   2.6s


[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:  3.1min finished


GridSearchCV(cv=4, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': [50, 80, 120], 'max_features': ['auto', 'sqrt', 'log2'], 'min_samples_leaf': [5, 10, 20], 'n_estimators': [100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

In [31]:
random_search.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=50, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [33]:
m = RandomForestClassifier(n_estimators=500, min_samples_leaf=10, max_depth=30,\
                          max_features=60)
m.fit(X_train, y_train)
auc(m, X_train, X_val)

(0.9866931335903361, 0.9649259132908433)

In [ ]:
m.fit(X,y)
pred = m.predict_proba(test)

In [ ]:
submit = pd.DataFrame()
submit['test_id'] = id
submit['is_female'] = pred[:,1]
submit.to_csv("submissionrf.csv", index=False)